A simple file to do some debugging on the Python/Rust interface.

In [1]:
import ilqr
import numpy as np

In [23]:
Q = np.array([[0, 0], [0, 0]])
Qf = np.array([[1, 0], [0, 1]])
R = np.array([[1e-5]])
s = ilqr.ILQRSolver(2, 1, Q, Qf, R)
print(s)

ILQRSolver { state_dim: 2, control_dim: 1, Q: VecStorage { data: [0.0, 0.0, 0.0, 0.0], nrows: Dyn(2), ncols: Dyn(2) }, Qf: VecStorage { data: [1.0, 0.0, 0.0, 1.0], nrows: Dyn(2), ncols: Dyn(2) }, R: VecStorage { data: [1e-5], nrows: Dyn(1), ncols: Dyn(1) } }


In [26]:
s.solve(np.zeros(2), np.ones(2), lambda x, u: x, time_steps=10)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [25]:
s.solve(np.zeros(2), np.ones(2), lambda x, u: x, time_steps=10, full_output=True)

{'control': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'time_taken': 0.000235441,
 'it_taken': 0,
 'gradient_norm': [0.0],
 'cost': [2.0],
 'kind': 'converged'}

In [4]:
s.solve(np.zeros(2), np.ones(2), lambda x, u: x, time_steps=10, jacobians=lambda x, u: (np.zeros((2, 2)), np.zeros((2, 1))))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

# Error Testing


Add tag `raises-exception` to cells that fail.

In [5]:
ilqr.ILQRSolver(1, 1, np.asarray([[0, 0]]),
                np.asarray([[1]]), np.asarray([[1e-5]]))

ValueError: Invalid state cost matrix dimension; expected 1x1 but got 1x2.

In [6]:
ilqr.ILQRSolver(1, 1, np.asarray([[0], [0]]),
                np.asarray([[1]]), np.asarray([[1e-5]]))

ValueError: Invalid state cost matrix dimension; expected 1x1 but got 2x1.

In [7]:
ilqr.ILQRSolver(1, 1, np.asarray([[0]]),
                np.asarray([[1, 0]]), np.asarray([[1e-5]]))

ValueError: Invalid final state cost matrix dimension; expected 1x1 but got 1x2.

In [8]:
ilqr.ILQRSolver(1, 1, np.asarray([[0]]),
                np.asarray([[1], [0]]), np.asarray([[1e-5]]))

ValueError: Invalid final state cost matrix dimension; expected 1x1 but got 2x1.

In [9]:
ilqr.ILQRSolver(1, 1, np.asarray([[0]]),
                np.asarray([[1]]), np.asarray([[1e-5, 0]]))

ValueError: Invalid control cost matrix dimension; expected 1x1 but got 1x2.

In [10]:
ilqr.ILQRSolver(1, 1, np.asarray([[0]]),
                np.asarray([[1]]), np.asarray([[1e-5], [0]]))

ValueError: Invalid control cost matrix dimension; expected 1x1 but got 2x1.

In [11]:
s = ilqr.ILQRSolver(1, 1, np.asarray([[0]]), np.asarray([[1]]), np.asarray([[1e-5]]))
s

ILQRSolver { state_dim: 1, control_dim: 1, Q: VecStorage { data: [0.0], nrows: Dyn(1), ncols: Dyn(1) }, Qf: VecStorage { data: [1.0], nrows: Dyn(1), ncols: Dyn(1) }, R: VecStorage { data: [1e-5], nrows: Dyn(1), ncols: Dyn(1) } }

In [12]:
s.solve(np.zeros(2), np.ones(1), lambda x, u:x, time_steps=1)

ValueError: Invalid initial state dimension; expected 1 but got 2.

In [13]:
s.solve(np.zeros(1), np.ones(2), lambda x, u: x, time_steps=1)

ValueError: Invalid target dimension; expected 1 but got 2.

In [14]:
s.solve(np.zeros(1), np.ones(1), lambda x, u: x, time_steps=0)

ValueError: Time steps must non-zero.

In [15]:
s.solve(np.zeros(1), np.ones(1), lambda x, u: x, time_steps=1, initialization=-1.0)

ValueError: Initialization standard deviation must be non-negative.

In [16]:
s.solve(np.zeros(1), np.ones(1), None, time_steps=1)

TypeError: 'NoneType' object is not callable

In [17]:
s.solve(np.zeros(1), np.ones(1), lambda x:x, time_steps=1)

TypeError: <lambda>() takes 1 positional argument but 2 were given

In [18]:
def f1(x, u):
    raise OSError("Why not ?")


s.solve(np.zeros(1), np.ones(1), f1, time_steps=1)

OSError: Why not ?

In [19]:
def f2(x, u):
    return "Why not too ?"


s.solve(np.zeros(1), np.ones(1), f2, time_steps=1)

TypeError: Can't extract `str` to `Vec`

In [20]:
def f3(x, u):
    return [float('nan')]


s.solve(np.zeros(1), np.ones(1), f3, time_steps=1)

RuntimeError: Instable problem - NaN detected in the control sequence. Choose a small gradient clipping value, or reduce the number of iterations. (iteration: 0)

In [21]:
def f4(x, u):
    return [0]


s = ilqr.ILQRSolver(1, 1, np.asarray(
    [[0]]), np.asarray([[0]]), np.asarray([[0]]))
s.solve(np.zeros(1), np.ones(1), f4, time_steps=1)

RuntimeError: The Quu matrix should be invertible.